<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson2/04_refactor_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson2} -->

# Refactor of Baseline Model
<!--- @wandbcode{course-lesson2} -->

The same code as in the previous notebook, but refactored into a small functions. This will be useful for the hyperparameter tuning.

In [1]:
import wandb
import pandas as pd
import torchvision.models as tvmodels
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

/root/miniconda3/envs/course/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    arch="resnet18",
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
    log_preds=True,
)

In [3]:
def download_data():
    processed_data_at = wandb.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
    processed_dataset_dir = Path(processed_data_at.download())
    return processed_dataset_dir

In [4]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [5]:
def get_df(processed_dataset_dir, is_test=False):
    df = pd.read_csv(processed_dataset_dir / 'data_split.csv')
    
    if not is_test:
        df = df[df.Stage != 'test'].reset_index(drop=True)
        df['is_valid'] = df.Stage == 'valid'
    else:
        df = df[df.Stage == 'test'].reset_index(drop=True)
        
    
    # assign paths
    df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
    df["label_fname"] = [label_func(f) for f in df.image_fname.values]
    return df

In [6]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [7]:
def log_predictions(learn):
    "Log a Table with model predictions"
    samples, outputs, predictions = get_predictions(learn)
    table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
    wandb.log({"pred_table":table})

In [8]:
def log_final_metrics(learn):
    scores = learn.validate()
    metric_names = ['final_loss'] + [f'final_{x.name}' for x in learn.metrics]
    final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
    for k,v in final_results.items(): 
        wandb.summary[k] = v

In [9]:
def train(config):
    set_seed(config.seed, reproducible=True)
    run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=config)

    config = wandb.config
    
    processed_dataset_dir = download_data()
    df = get_df(processed_dataset_dir)
    
    dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

    metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
               TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

    learn = unet_learner(dls, arch=getattr(tvmodels, config.arch), pretrained=config.pretrained, metrics=metrics)

    callbacks = [
        SaveModelCallback(monitor='miou'),
        WandbCallback(log_preds=False, log_model=True)
    ]

    learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)
    
    if config.log_preds:
        log_predictions(learn)
        
    log_final_metrics(learn)

    wandb.finish()

## Run the training

In [10]:
train(train_config)

wandb: Currently logged in as: nov05 (novemberfifth). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:15.2
/root/miniconda3/envs/course/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/course/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.510675,0.393148,0.311786,0.866503,0.757810,0.000000,0.000000,0.000000,0.558189,0.000000,01:24
1,0.485744,0.330940,0.323232,0.879704,0.774036,0.000000,0.000000,0.000000,0.608883,0.000000,01:24
2,0.345399,0.297727,0.337490,0.892551,0.800686,0.000000,0.000000,0.000000,0.669189,0.000000,01:31
3,0.301314,0.291701,0.341025,0.899816,0.818028,0.000000,0.000000,0.000000,0.669332,0.000000,01:25
4,0.283175,0.259774,0.347788,0.906870,0.818020,0.000000,0.000000,0.000000,0.709626,0.000000,01:24
5,0.257303,0.254343,0.348690,0.908845,0.827963,0.000000,0.000000,0.000000,0.704020,0.000000,01:26
6,0.220724,0.264321,0.354786,0.907718,0.822068,0.082513,0.000000,0.000000,0.671207,0.000000,01:30
7,0.210437,0.225861,0.369055,0.917812,0.843039,0.082020,0.000000,0.000000,0.740516,0.000000,01:28
8,0.188992,0.233227,0.373478,0.918577,0.842969,0.104457,0.001937,0.000000,0.746402,0.000000,01:28
9,0.178291,0.225700,0.373360,0.919753,0.845918,0.097891,0.001070,0.000000,0.748886,0.000000,01:27


Better model found at epoch 0 with miou value: 0.3117860126672006.
Better model found at epoch 1 with miou value: 0.3232319641223874.
Better model found at epoch 2 with miou value: 0.33748954874273907.
Better model found at epoch 3 with miou value: 0.34102517341586774.
Better model found at epoch 4 with miou value: 0.34778791967703265.
Better model found at epoch 5 with miou value: 0.3486897515412762.
Better model found at epoch 6 with miou value: 0.3547864854691068.
Better model found at epoch 7 with miou value: 0.36905527285522105.
Better model found at epoch 8 with miou value: 0.37347750862874207.


background_iou,▁▃▄▅▆▇▆███
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
miou,▁▂▄▄▅▅▆▇██
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████


**Issue (solved)**
```
Could not load library libcudnn_cnn_infer.so.8. Error: libcuda.so: cannot open shared object file: No such file or directory
Please make sure libcudnn_cnn_infer.so.8 is in your library path!
```
https://discuss.pytorch.org/t/libcudnn-cnn-infer-so-8-library-can-not-found/164661
> Hi, I also faced the same problem on WSL2.  
> As discussed in this issue 155 it is just a matter to add this in the `.bashrc`:  
> **export LD_LIBRARY_PATH=/usr/lib/wsl/lib:$LD_LIBRARY_PATH**  
> Be sure that your library is in /usr/lib/wsl/lib, to see it you can run  
> **ldconfig -p | grep cuda**  